# Correlation coefficient amidst SST and SPEI


In [80]:
import xarray as xr
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import pearsonr, linregress
import matplotlib.pyplot as plt

In [14]:
spei_country = pd.read_excel('/Users/kris/GitHub/fall_2024_trp_proj/data/SPEI_AVG_CROP_ZC.xlsx', 'Country')
ncdata =  xr.open_dataset('/Users/kris/GitHub/fall_2024_trp_proj/data/sst.mnmean.nc')

In [167]:
ncdata

<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, time: 2048, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2024-08-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 ...
    sst        (time, lat, lon) float32 ...
Attributes: (12/37)
    climatology:               Climatology is based on 1971-2000 SST, Xue, Y....
    description:               In situ data: ICOADS2.5 before 2007 and NCEP i...
    keywords_vocabulary:       NASA Global Change Master Directory (GCMD) Sci...
    keywords:                  Earth Science > Oceans > Ocean Temperature > S...
    instrument:                Conventional thermometers
    source_comment:            SSTs were observed by conventional thermometer...
    ...                        ...
    creator_url_original:      https://www.ncei.noaa.gov
    license:                   No constraints on data access or use
    comment:                   SSTs were observed by conventional thermometer...
    summary:                   ERSST.v5 is developed based on v4 after revisi...
    dataset_title:             NOAA Extended Reconstructed SST V5
    data_modified:             2024-09-04

In [127]:
spei_country['time'] = pd.to_datetime(spei_country['time'], format='%Y-%m-%d')

In [129]:
px.line(spei_country, x='time', y='Indonesia_1')
px.line(spei_country, x='time', y='Indonesia_3')
px.line(spei_country, x='time', y='Indonesia_6')

In [136]:
sst = ncdata['sst'].sel(time = slice('1980-01-01', '2023-12-01'))
sst

<xarray.DataArray 'sst' (time: 528, lat: 89, lon: 180)>
[8458560 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 88.0 86.0 84.0 82.0 80.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2023-12-01
Attributes:
    long_name:     Monthly Means of Sea Surface Temperature
    units:         degC
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    dataset:       NOAA Extended Reconstructed SST V5
    parent_stat:   Individual Values
    actual_range:  [-1.8     42.32636]
    valid_range:   [-1.8 45. ]

In [137]:
# Calculate the correlation for each grid cell in SST
latitudes = sst['lat'].values
longitudes = sst['lon'].values
correlation_map = np.zeros((len(latitudes), len(longitudes)))

In [160]:
px.imshow(sst.sel(time = '1980-01-01').values)

In [163]:
spei_array  = spei_country['Indonesia_6'].values

In [164]:
# Iterate over the SST grid cells and calculate correlation with SPEI
for i, lat in enumerate(latitudes):
    for j, lon in enumerate(longitudes):
        sst_series = sst[:, i, j].values  # SST time series for this grid cell
        
        # Ensure valid data for correlation (no NaNs)
        mask = ~np.isnan(spei_array) & ~np.isnan(sst_series)
        if np.sum(mask) > 0:
            correlation_map[i, j], _ = pearsonr(sst_series[mask], spei_array[mask])
        else:
            correlation_map[i, j] = np.nan  # Assign NaN if not enough data

/var/folders/m_/4k615z_x1ws9cfblht3sczn40000gn/T/ipykernel_83908/1567123549.py:9: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



In [165]:
px.imshow(correlation_map)